In [ ]:
import clf as clf
import estimator as estimator
#imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectKBest
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn import __all__, svm
from sklearn.model_selection import StratifiedKFold


In [ ]:
#get csv file and convert to dataframe
virus_df = pd.read_csv("hw5_data.csv", sep=',')
virus_df.head()

In [ ]:
y = virus_df['target']
X = virus_df.drop('target', axis=1)

In [ ]:
#split dataframe two part, target and feature
lr = LogisticRegression(solver='liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 0)

In [ ]:
#remove highly correlated features
correlated_features = set()
correlation_matrix = virus_df.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

X_train.drop(labels=correlated_features, axis=1, inplace=True)
X_test.drop(labels=correlated_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((3749, 1799), (1250, 1799))

In [ ]:
#cross validation and test score of logistic regression
ss = StandardScaler()
X_train_scale = ss.fit_transform(X_train)
print("LR Cross Validation Mean Accuracy: ", cross_val_score(lr, X_train_scale, y_train, cv=5).mean())
lr.fit(X_train_scale, y_train)


X_test_scale = ss.fit_transform(X_test)
print('Test score: ', sum(lr.predict(X_test_scale)==y_test)/len(y_test) )

LR Cross Validation Mean Accuracy:  0.514808722741433
Test score:  0.5168



## Hyperparameter Search (Grid Search) and feature selections methods applying on variety algorithms 

In [ ]:
#Random forest classifier with grid search and feature selection (RFE)
pipe_rfe_rfc =  Pipeline([
    ('rfe', RFE(estimator=RandomForestClassifier())),
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
])


# Create the parameter grid
param_grid_rfc = [{
    'clf__max_depth': [2, 3, 4],
    'clf__max_features': [2, 3, 4, 5, 6]
}]
#
# Create an instance of GridSearch Cross-validation estimator
#
grid_rfe_rfc = GridSearchCV(estimator=pipe_rfe_rfc,
                     param_grid = param_grid_rfc,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)

# Train the RandomForestClassifier
grid_rfe_rfc = grid_rfe_rfc.fit(X_train, y_train)

# Print the training score of the best model
print(grid_rfe_rfc.best_score_)

# Print the model parameters of the best model
#print(grid_rfe_rfc.best_params_)

# Print the test score of the best model
clfRFC = grid_rfe_rfc.best_estimator_
print('Test accuracy: %.3f' % clfRFC.score(X_test, y_test))


validation and test set performances with LDA

In [ ]:
lda = LDA(n_components=4)
lda.fit(X_train_scale, y_train)
X_train_lda = lda.transform(X_train_scale)

In [ ]:
# pipeline for training a logistic regression classifier using linear discriminant analysis (LDA).
pipe_lda_lr= Pipeline([('lda', LDA()),
              ('model', LogisticRegression(solver='liblinear', max_iter=100000), 'scaler', StandardScaler())
             ])

params = {'model__C': [0.1, 1, 10], 'lda__n_components': [2, 3, 4]}

grid_lda_lr = GridSearchCV(pipe_lda_lr, param_grid=params, scoring='accuracy', cv=5, refit=True)
grid_lda_lr.fit(X_train_scale, y_train)

#print("Best parameters: {}".format(grid_lda_lr.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_lda_lr.best_score_))
print('Test score:' )
print( grid_lda_lr.score(X), y)
print("Best estimator:\n{}".format(grid_lda_lr.best_estimator_))

In [ ]:
#from https://makeabilitylab.github.io/physcomp/signals/FeatureSelectionAndHyperparameterTuning/index.html

#Defining a pipeline for training a classifier using a combination of feature scaling, feature selection, and classification.
scaler = StandardScaler()
feature_selector = SelectKBest(k=5)
pipeline = Pipeline([('scaler', scaler), ('selector', feature_selector), ('estimator', clf)])

# Setup our hyperparameter ranges for the grid search.
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)

# The hierarchy in the pipeline can be reached using double underscore(‘__’)
scaler = StandardScaler()
feature_selector = SelectKBest(k=5)
pipeline = Pipeline([('scaler', scaler), ('selector', feature_selector), ('estimator', clf)])

# Setup our hyperparameter ranges for the grid search.
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)

# The hierarchy in the pipeline can be reached using double underscore(‘__’)
param_grid = [
  {'selector__k': [4, 5], 'estimator__degree': [1, 2, 3], 'estimator__gamma': gamma_range, 'estimator__kernel': ['poly']},
  {'estimator__C': C_range, 'estimator__kernel': ['linear']},
  {'estimator__C': C_range, 'estimator__gamma': gamma_range, 'estimator__kernel': ['rbf']},
 ]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
clf = svm.SVC()
grid_search = GridSearchCV(pipeline, param_grid, cv=skf, return_train_score=True)
grid_search.fit(X, y)

#print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print('Test score:' )
print( grid_search.score(X), y)
print("Best estimator:\n{}".format(grid_search.best_estimator_))


In [ ]:
from sklearn.feature_selection import RFE

# Define a pipeline that consists of RFE and a logistic regression classifier
pipe_rfe_lr = Pipeline([('rfe', RFE(estimator=LogisticRegression(max_iter=1000000))), ('scaler', StandardScaler()),
                    ('clf', LogisticRegression(max_iter=100000))])

# Use grid search to fine-tune the hyperparameters of the logistic regression classifier
param_grid = {'rfe__estimator__C': [0.1, 1, 10],
              'clf__C': [0.1, 1, 10]}
grid_rfe_lr = GridSearchCV(pipe_rfe_lr, param_grid, cv=10)
grid_rfe_lr.fit(X_train_scale, y_train)


#print("Best parameters: {}".format(grid_rfe_lr.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_rfe_lr.best_score_))
print('Test score:' )
print( grid_rfe_lr.score(X), y)


In [ ]:
# Define a pipeline that consists of RFE and a decision tree classifier
# create pipeline
pipe_rfe_dt = Pipeline([('rfe', RFE(estimator=DecisionTreeClassifier())),
                    ('clf', DecisionTreeClassifier())])

# Use grid search to fine-tune the hyperparameters of the decision tree classifier
param_grid = {'rfe__estimator__criterion': ['gini', 'entropy'],
              'clf__criterion': ['gini', 'entropy'],
              'clf__min_samples_leaf': [1, 2, 3],
    'clf__max_depth': [1, 2, 3]}



grid_rfe_dt = GridSearchCV(pipe_rfe_dt, param_grid, cv=10)
grid_rfe_dt.fit(X_train, y_train)

#print("Best parameters: {}".format(grid_rfe_dt.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_rfe_dt.best_score_))
print('Test score:' )
print( grid_rfe_dt.score(X), y)


In [ ]:
#Grid search and the GenericUnivariateSelect filter method to perform feature selection on a logistic regression classifier
select = GenericUnivariateSelect(score_func=mutual_info_classif, mode="k_best", param=5)
pipe_unisel_lr = Pipeline([
      ("MutualInfo", select),
      ("LR", LogisticRegression(solver='liblinear', max_iter=100000), 'scaler', StandardScaler() )
])

param_grid = {'LR__C': [0.1, 1, 10]}

grid_unisel_lr = GridSearchCV(pipe_unisel_lr, param_grid, cv=10)
grid_unisel_lr.fit(X_train_scale, y_train)

#print("Best parameters: {}".format(grid_unisel_lr.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_unisel_lr.best_score_))
print('Test score:' )
print( grid_unisel_lr.score(X), y)

In [ ]:
#from https://vitalflux.com/grid-search-explained-python-sklearn-examples/
from sklearn.pipeline import make_pipeline

#Grid search and recursive feature elimination (RFE) to perform feature selection and hyperparameter tuning on a support vector classifier
pipe_rfe_svc = Pipeline([('rfe', RFE(estimator=SVC(random_state=1))),
                    ('scaler', StandardScaler()),
                    ('clf', SVC(random_state=1))])

# Define a pipeline that consists of RFE and a logistic regression classifier
param_grid = [{                    'clf__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'clf__kernel': ['linear']
                  },
                 {
                    'clf__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'clf__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
                    'clf__kernel': ['rbf']
                 }]

# Create an instance of GridSearch Cross-validation estimator
#
grid_rfe_svc = GridSearchCV(pipe_rfe_svc,
                     param_grid = param_grid,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)

grid_rfe_svc.fit(X_train, y_train)

#print("Best parameters: {}".format(grid_rfe_svc.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_rfe_svc.best_score_))
print('Test score:' )
print( grid_rfe_svc.score(X), y)


In [ ]:
#Grid search to perform feature selection and hyperparameter tuning on a k-nearest neighbors (KNN) classifier
pipe_kbest_kneigh =Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=5)),
                ('classifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=3, p=2,
                                      weights='distance'))],verbose=False)

param_grid = [{'selector__k': [5, 10, 20, 30]},
              {'classifier': [KNeighborsClassifier()],'classifier__n_neighbors': [3, 7, 11],
              'classifier__weights': ['uniform', 'distance']}]

grid_k_best_kneigh = GridSearchCV(pipe_kbest_kneigh, param_grid, cv=10, verbose=0)
grid_k_best_kneigh.fit(X, y)
#print("Best parameters: {}".format(grid_k_best_kneigh.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_k_best_kneigh.best_score_))
print('Test score:' )
print( grid_k_best_kneigh.score(X), y)


# Ensemble

Majority Voting

In [78]:

clf1 = LogisticRegression(penalty='l2', C=0.001, solver='lbfgs', random_state=0, max_iter=100000)
clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=1)
clf3 = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')

pipe1 = Pipeline([('scaler', StandardScaler()),('logreg', clf1)])
pipe2 = Pipeline([('scaler', StandardScaler()),('tree', clf2)])
pipe3 = Pipeline([('scaler', StandardScaler()),('knn', clf3)])

labs = ['Logistic Regression', 'Decision Tree', 'k-Nearest Neighbors']
clfs = [pipe1, pipe2, pipe3]
clfs = zip(labs, clfs)

for lab, clf in clfs:
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=10, scoring='accuracy')
    print(f'Accuracy {scores.mean():.2f} (+/- {scores.std():.2f}) [{lab}]')


Accuracy 0.60 (+/- 0.01) [Logistic Regression]
Accuracy 0.47 (+/- 0.00) [Decision Tree]
Accuracy 0.24 (+/- 0.03) [k-Nearest Neighbors]


In [79]:
ems = [('lr', pipe1),('dt', pipe2),('knn', pipe3)]
clf4 = VotingClassifier(estimators= ems, weights=None, voting='soft')

scores = cross_val_score(estimator=clf4, X=X_train, y=y_train, cv=5, scoring='accuracy')
print(f'Accuracy {scores.mean():.2f} (+/- {scores.std():.2f}) [Ensemble]')

Accuracy 0.40 (+/- 0.01) [Ensemble]


Bagging

In [83]:

classifiers = []
names = []

classifiers.append( VotingClassifier([ (str(x), DecisionTreeClassifier() ) for x in range(20) ]) )
names.append("Vote(DTx20)")

classifiers.append( VotingClassifier([ (str(x), LogisticRegression(solver='liblinear', multi_class='auto', max_iter=100000) ) for x in range(20)  ]) )
names.append("Vote(LRx20)")

classifiers.append( VotingClassifier([ (str(x), RandomForestClassifier( oob_score=True) ) for x in range(20)  ]) )
names.append("Vote(RFx20)")

classifiers.append( VotingClassifier([ (str(x), KNeighborsClassifier() ) for x in range(20) ]) )
names.append("Vote(NNx20)")

classifiers.append( BaggingClassifier(DecisionTreeClassifier(), n_estimators=20, max_samples=0.6, oob_score=True) )
names.append("Bag(DTx20)")

classifiers.append( BaggingClassifier(RandomForestClassifier(n_estimators=20, oob_score=True)) )
names.append("Bag(RFx20)")

classifiers.append( BaggingClassifier(LogisticRegression(solver='liblinear', multi_class='auto', max_iter=100000), n_estimators=20, max_samples=0.6, oob_score=True) )
names.append("Bag(LRx20)")
classifiers.append( BaggingClassifier(KNeighborsClassifier(), n_estimators=20, max_samples=0.6, oob_score=True) )
names.append("Bag(NNx20)")


print("virus", ":")
for model, m_name in zip(classifiers, names):
    model.fit(X_train, y_train)
    print("\t", m_name, accuracy_score(y_test, model.predict(X_test)))
    if "oob_score_" in dir(model):
      print("\t\tOOB:", model.oob_score_)


virus :
	 Vote(DTx20) 0.8488
	 Vote(LRx20) 0.6008
	 Vote(RFx20) 0.9064
	 Vote(NNx20) 0.7464
	 Bag(DTx20) 0.908
		OOB: 0.8941050946919178


C:\Users\nilsu\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\ensemble\_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,
C:\Users\nilsu\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\ensemble\_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,
C:\Users\nilsu\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\ensemble\_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,
C:\Users\nilsu\PycharmProjects\pythonProject11\venv\lib\site-packages\sklearn\ensemble\_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


	 Bag(RFx20) 0.9064
	 Bag(LRx20) 0.6672
		OOB: 0.6521739130434783
	 Bag(NNx20) 0.7688
		OOB: 0.7444651907175247


I aimed this in this code, finding cross validation scores of searching and classifier methods thus bigger cross validation scores will be good for our classification of dataset. In the ensemble methods, bigger accuracy value of classification method is better for our dataset.
